# SET UP:

In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import re

# Loading the training data:

In [2]:
train_list=os.listdir("S:/eDU/TopDocs/DLProjects/DRDO/Dataset/DukeMTMC-reID/DukeMTMC-reID/bounding_box_train")

l=len(train_list)
print(l)

traindf=pd.DataFrame(train_list)

train_label=[]
for n in train_list:
    arr=re.split(r'_',n)
    train_label.append(arr[0])    

#trainldf=pd.DataFrame(train_label)
traindf['PersonID']=train_label
traindf.columns= ['ImageName','PersonID']

print(traindf)

traindf.to_csv('trainPersonID.csv', index=True)

16522
                  ImageName PersonID
0      0001_c2_f0046182.jpg     0001
1      0001_c2_f0046302.jpg     0001
2      0001_c2_f0046422.jpg     0001
3      0001_c2_f0046542.jpg     0001
4      0001_c2_f0046662.jpg     0001
...                     ...      ...
16517  7140_c4_f0175508.jpg     7140
16518  7140_c4_f0175628.jpg     7140
16519  7140_c4_f0175748.jpg     7140
16520  7140_c4_f0175868.jpg     7140
16521  7140_c4_f0175988.jpg     7140

[16522 rows x 2 columns]


# Loading the Validation data:

In [3]:
val_list=os.listdir("S:/eDU/TopDocs/DLProjects/DRDO/Dataset/DukeMTMC-reID/DukeMTMC-reID/query")

l=len(val_list)
l

valdf=pd.DataFrame(val_list)

val_label=[]
for n in val_list:
    arr=re.split(r'_',n)
    val_label.append(arr[0])
    
    
valdf['PersonID']=val_label
valdf.columns= ['ImageName','PersonID']
print(valdf)

                 ImageName PersonID
0     0005_c2_f0046985.jpg     0005
1     0005_c5_f0051781.jpg     0005
2     0005_c6_f0030883.jpg     0005
3     0019_c1_f0051470.jpg     0019
4     0019_c2_f0055712.jpg     0019
...                    ...      ...
2223  6535_c6_f0099299.jpg     6535
2224  6535_c7_f0050473.jpg     6535
2225  6535_c8_f0005374.jpg     6535
2226  7139_c1_f0156370.jpg     7139
2227  7139_c2_f0160575.jpg     7139

[2228 rows x 2 columns]


from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

folder='S:/eDU/TopDocs/DLProjects/DRDO/Dataset/DukeMTMC-reID/DukeMTMC-reID/bounding_box_train'

for count, file in listdir(folder):

# Storing the weights of ResNet50(after making the top layer false) into base_model:

In [4]:
base_model = tf.keras.applications.ResNet50(input_shape=(224,224,3),weights='imagenet', include_top = False)
print(base_model.summary())

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

# Checking the Names of different layers of the ResNet50 stored into our base_model:

In [5]:
for i, layer in enumerate(base_model.layers):
  print(i, layer.name)

0 input_1
1 conv1_pad
2 conv1_conv
3 conv1_bn
4 conv1_relu
5 pool1_pad
6 pool1_pool
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_conv
11 conv2_block1_2_bn
12 conv2_block1_2_relu
13 conv2_block1_0_conv
14 conv2_block1_3_conv
15 conv2_block1_0_bn
16 conv2_block1_3_bn
17 conv2_block1_add
18 conv2_block1_out
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_conv
23 conv2_block2_2_bn
24 conv2_block2_2_relu
25 conv2_block2_3_conv
26 conv2_block2_3_bn
27 conv2_block2_add
28 conv2_block2_out
29 conv2_block3_1_conv
30 conv2_block3_1_bn
31 conv2_block3_1_relu
32 conv2_block3_2_conv
33 conv2_block3_2_bn
34 conv2_block3_2_relu
35 conv2_block3_3_conv
36 conv2_block3_3_bn
37 conv2_block3_add
38 conv2_block3_out
39 conv3_block1_1_conv
40 conv3_block1_1_bn
41 conv3_block1_1_relu
42 conv3_block1_2_conv
43 conv3_block1_2_bn
44 conv3_block1_2_relu
45 conv3_block1_0_conv
46 conv3_block1_3_conv
47 conv3_block1_0_bn
48 conv3_block1_3_bn

# Add top Layers according to the number of classes of our dataset:

In [6]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(200,kernel_regularizer=tf.keras.regularizers.l2(0.0001),activation="relu")(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
preds = tf.keras.layers.Dense(702, activation ='softmax')(x)

# Checking the summary or details of our model after adding the top layers to the base_model:

In [7]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=preds)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

# Checking the Names of different layers of our model after adding the top layer to ResNet50:

In [8]:
for i, layer in enumerate(model.layers):
  print(i, layer.name)

0 input_1
1 conv1_pad
2 conv1_conv
3 conv1_bn
4 conv1_relu
5 pool1_pad
6 pool1_pool
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_conv
11 conv2_block1_2_bn
12 conv2_block1_2_relu
13 conv2_block1_0_conv
14 conv2_block1_3_conv
15 conv2_block1_0_bn
16 conv2_block1_3_bn
17 conv2_block1_add
18 conv2_block1_out
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_conv
23 conv2_block2_2_bn
24 conv2_block2_2_relu
25 conv2_block2_3_conv
26 conv2_block2_3_bn
27 conv2_block2_add
28 conv2_block2_out
29 conv2_block3_1_conv
30 conv2_block3_1_bn
31 conv2_block3_1_relu
32 conv2_block3_2_conv
33 conv2_block3_2_bn
34 conv2_block3_2_relu
35 conv2_block3_3_conv
36 conv2_block3_3_bn
37 conv2_block3_add
38 conv2_block3_out
39 conv3_block1_1_conv
40 conv3_block1_1_bn
41 conv3_block1_1_relu
42 conv3_block1_2_conv
43 conv3_block1_2_bn
44 conv3_block1_2_relu
45 conv3_block1_0_conv
46 conv3_block1_3_conv
47 conv3_block1_0_bn
48 conv3_block1_3_bn

for layer in model.layers[:175]:
  layer.trainable = False

for layer in model.layers[175:]:
  layer.trainable = True

# PreProcessing the Train and Validation data

In [10]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

In [11]:
train_generator = train_datagen.flow_from_dataframe(traindf,
                                                   'S:/eDU/TopDocs/DLProjects/DRDO/Dataset/DukeMTMC-reID/DukeMTMC-reID/bounding_box_train',
                                                   x_col='ImageName',y_col='PersonID',
                                                   target_size = (224, 224),
                                                   color_mode = 'rgb',
                                                   batch_size =128, 
                                                   class_mode = 'categorical')

validation_generator = val_datagen.flow_from_dataframe(
    valdf, 
    'S:/eDU/TopDocs/DLProjects/DRDO/Dataset/DukeMTMC-reID/DukeMTMC-reID/query', 
    x_col='ImageName',
    y_col='PersonID',
    color_mode = 'rgb',
    target_size=(224,224),
    class_mode='categorical',
    batch_size=128)

Found 16522 validated image filenames belonging to 702 classes.
Found 2228 validated image filenames belonging to 702 classes.


In [12]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=11,validation_data=validation_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 130 steps, validate for 18 steps
Epoch 1/11


acc = history.history['accuracy']
loss = history.history['loss']

plt.figure()
plt.plot(acc, label='Training Accuracy')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')

plt.figure()

plt.plot(loss, label='Training Loss')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from PIL import Image

def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')
   return np_image

Sample_Image=load('S:/eDU/TopDocs/DLProjects/DRDO/Dataset/DukeMTMC-reID/DukeMTMC-reID/bounding_box_train/0008_c3_f0026678.jpg')
Sample_Image = tf.keras.preprocessing.image.img_to_array(Sample_Image)


Sample_Image = np.expand_dims(Sample_Image, axis = 0)


Sample_Image = tf.keras.applications.resnet50.preprocess_input(Sample_Image)
predictions = model.predict(Sample_Image)
print('Predictions:', max(max(predictions))))

In [ ]:
score=0
for i in range(len(predictions)):
  if np.argmax(predictions[i]) == train_label[i]:
    score+=1

rank1 = score/float(len(train_label))

rank1

In [ ]:
score=0
for i in range(len(predictions)):
  if train_label[i] in np.argsort(predictions)[::-1][:5]:
    score+=1

rank5 = score/float(len(train_label))

# LEARNINGS:-

# To solve for jumping-around

A larger batch size will give you a better gradient and will help to prevent jumping around
You may also want to consider a smaller learning rate, or a learning rate scheduler (or decay) to allow the network to "settle in" as it trains
Implications of a larger batch-size

Too large of a batch_size can produce memory problems, especially if you are using a GPU. Once you exceed the limit, dial it back until it works. This will help you find the max batch-size that your system can work with.
Too large of a batch size can get you stuck in a local minima, so if your training get stuck, I would reduce it some. Imagine here you are over-correcting the jumping-around and it's not jumping around enough to further minimize the loss function.
When to reduce epochs

If your train error is very low, yet your test/validation is very high, then you have over-fit the model with too many epochs.
The best way to find the right balance is to use early-stopping with a validation test set. Here you can specify when to stop training, and save the weights for the network that gives you the best validation loss. (I highly recommend using this always)
When to adjust steps-per-epoch

Traditionally, the steps per epoch is calculated as train_length // batch_size, since this will use all of the data points, one batch size worth at a time.
If you are augmenting the data, then you can stretch this a tad (sometimes I multiply that function above by 2 or 3 etc. But, if it's already training for too long, then I would just stick with the traditional approach.

Batch_size determines the number of samples in each mini batch. Its maximum is the number of all samples, which makes gradient descent accurate, the loss will decrease towards the minimum if the learning rate is small enough, but iterations are slower. 

Its minimum is 1, resulting in stochastic gradient descent: Fast but the direction of the gradient step is based only on one example, the loss may jump around. batch_size allows to adjust between the two extremes: accurate gradient direction and fast iteration. Also, the maximum value for batch_size may be limited if your model + data set does not fit into the available (GPU) memory.

Steps_per_epoch the number of batch iterations before a training epoch is considered finished. If you have a training set of fixed size you can ignore it but it may be useful if you have a huge data set or if you are generating random data augmentations on the fly, i.e. if your training set has a (generated) infinite size. If you have the time to go through your whole training data set I recommend to skip this parameter.

Validation_steps similar to steps_per_epoch but on the validation data set instead on the training data. If you have the time to go through your whole validation data set I recommend to skip this parameter.